In [1]:
pwd

'/home/yjkim/new'

In [2]:

%%capture

# Package Import and Installation
import torch
import torch.nn as nn
import pandas as pd
import os

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from tqdm.notebook import tqdm


!pip3 install kobert-transformers
!pip3 install git+https://git@github.com/SKTBrain/KoBERT.git@master


In [3]:
from kobert_transformers import get_distilkobert_model, get_tokenizer

# Import Data


# CUDA
print("CUDA AVAILABLE: ", torch.cuda.is_available())
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer = get_tokenizer()
KoBERT    = get_distilkobert_model()

/home/yjkim/anaconda3/envs/allml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yjkim/anaconda3/envs/allml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yjkim/anaconda3/envs/allml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yjkim/anaconda3/envs/allml/lib/python3.7/site-

CUDA AVAILABLE:  True


In [4]:
# Need PAD_ID for data preprocessing
PAD_ID = tokenizer.pad_token_id
SEP_ID = tokenizer.sep_token_id
CLS_ID = tokenizer.cls_token_id

# Hyperparameters
hyperparams = {'num_epochs': 3,
               'batch_size': 32,
               'seq_len'   : 200,
               'learn_rate': 2e-5}

In [5]:
# Demonstration of KoBERT Tokenizer
ex_sentence = "본 모델은 영화 리뷰를 입력으로 하여 평점을 예측합니다."
ex_tokenize = tokenizer.tokenize(ex_sentence)
tokenized   = tokenizer.convert_tokens_to_ids(ex_tokenize)
untokenized = tokenizer.convert_ids_to_tokens(tokenized)
print(tokenized, '\n', untokenized)

[2408, 2046, 7086, 3394, 1900, 6431, 6116, 3840, 7078, 517, 7815, 4841, 7223, 3415, 7843, 54] 
 ['▁본', '▁모델', '은', '▁영화', '▁리', '뷰', '를', '▁입력', '으로', '▁', '하여', '▁평', '점을', '▁예측', '합니다', '.']


In [6]:
tokenizer.vocab_size

8002

In [7]:
class MovieDataset(Dataset):
    def __init__(self, tokenizer, input_file, label_file, seq_len=64):
        super().__init__()
        
        self.label_file = label_file
        self.seq_len = seq_len
        self.inputs  = open(input_file, 'r').read().splitlines()
        if label_file is not None:
            self.labels = open(label_file, 'r').read().splitlines()

        

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # Convert to tokens then augment tags
        tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(self.inputs[idx]))
        padded = [CLS_ID] + tokens[:self.seq_len-2] + [SEP_ID] + [PAD_ID] * (self.seq_len - len(tokens) - 2)
        length = len(tokens[:self.seq_len-2])
        if self.label_file is not None:
            labels = int(self.labels[idx])
            return {'inputs': torch.tensor(padded),
                    'labels': torch.tensor(labels),
                    'length': torch.tensor(length)}
        else:
            return {'inputs': torch.tensor(padded),
                    'length': torch.tensor(length)}

In [8]:
train_ds = MovieDataset(tokenizer, 'train_data', 'train_label', seq_len=hyperparams['seq_len'])
valid_ds = MovieDataset(tokenizer, 'valid_data', 'valid_label', seq_len=hyperparams['seq_len'])
test_ds  = MovieDataset(tokenizer, 'test_data', None, seq_len=hyperparams['seq_len'])

train_loader = DataLoader(train_ds, batch_size=hyperparams['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=128)
test_loader  = DataLoader(test_ds,  batch_size=128)

In [8]:
train_ds.inputs[1]

'불멸의 명작. 영화인을 꿈꾸는 사람이라면 반드시 봐야할 영화'

In [9]:
train_ds.labels[0]

'10'

In [10]:
train_ds[0]

{'inputs': tensor([   2, 2288, 7089,  517,  422,  453,  389,  708,  423,  389, 7095,  791,
         7206, 2734, 7069, 5468, 3376, 6493, 5561, 6903, 1390, 1597, 7543, 7828,
         4694, 3960, 5938, 2355, 6197, 4737, 7628, 7095, 5152, 6527, 6896,  517,
         7233, 7086, 5943, 7828, 3194, 7172, 7828, 4883, 4299, 5859,  517, 7145,
         7088, 6630, 3273,   54,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    

In [9]:
# Model
class MovieReviewModel(nn.Module):
    def __init__(self):
        super().__init__()
        # nn.CrossEntropyLoss in torch has built-in softmax. No need for activation.
        self.pretrained = KoBERT.to(DEVICE)
        self.linear     = nn.Linear(768, 10).to(DEVICE)
        
    def forward(self, x):
        # Pretrained Model Output -> (Embedding, Pooled)
        x, = self.pretrained(x)
        x  = self.linear(x[:,0])

        return x

model = MovieReviewModel()

# Loss
loss_func = nn.CrossEntropyLoss(ignore_index=PAD_ID)

# Optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=hyperparams['learn_rate'])

In [10]:
# Validating
def validate(model, valid_loader):
    model.eval()
    s, t = 0, 0
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            x = batch['inputs'].to(DEVICE)
            y = batch['labels'].cpu()
            q = batch['length']

            if type(model) == MovieReviewModel:
                z = model(x).cpu()
            else:
                z = model(x, q).cpu()
            p = torch.argmax(z, dim=-1).cpu() + 1

            s += torch.sum(y==p, [-1,0]).item()
            t += len(x)

        print('VALIDATION ACCURACY: %.2f %%' % (s / t * 100))

# Finetuning
def finetune(model, train_loader):
    model.train()
    t = 0
    for i, batch in enumerate(tqdm(train_loader)):
        x = batch['inputs'].to(DEVICE)
        y = batch['labels'].cpu() - 1
        z = model(x).cpu()

        l = loss_func(z, y)
        t += l.item()
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        # Print train loss, save model, and validate every 1000 steps
        if not (i % 1000):
            print('BATCH %d of %d TRAIN LOSS: %.3f' % (i, len(train_loader), l))
            print('SAVING MODEL ...')
            torch.save(model.state_dict(), 'checkpoint.pt')
            validate(model, valid_loader)
            model.train()

    print('TRAIN LOSS: %.3f' % t)



In [ ]:
# if os.path.exists('checkpoint.pt'):
#     model.load_state_dict(torch.load('checkpoint.pt'))
#     model.to(DEVICE)
# else:
 

for e in range(hyperparams['num_epochs']):
    print('******* EPOCH %d / %d ********' % (e+1, hyperparams['num_epochs']))
    finetune(model, train_loader)

******* EPOCH 1 / 3 ********


BATCH 0 of 281250 TRAIN LOSS: 2.370
SAVING MODEL ...



VALIDATION ACCURACY: 60.52 %
BATCH 1000 of 281250 TRAIN LOSS: 1.094
SAVING MODEL ...



VALIDATION ACCURACY: 76.94 %
BATCH 2000 of 281250 TRAIN LOSS: 1.585
SAVING MODEL ...



VALIDATION ACCURACY: 78.15 %
BATCH 3000 of 281250 TRAIN LOSS: 1.323
SAVING MODEL ...


...


VALIDATION ACCURACY: 82.75 %
BATCH 88000 of 281250 TRAIN LOSS: 0.874
SAVING MODEL ...



VALIDATION ACCURACY: 82.57 %
BATCH 89000 of 281250 TRAIN LOSS: 1.266
SAVING MODEL ...



VALIDATION ACCURACY: 82.52 %
BATCH 90000 of 281250 TRAIN LOSS: 1.242
SAVING MODEL ...



VALIDATION ACCURACY: 82.49 %
BATCH 91000 of 281250 TRAIN LOSS: 0.827
SAVING MODEL ...



VALIDATION ACCURACY: 82.69 %


In [11]:
def test(model, test_loader):
    model.eval()
    output = []
    for i, batch in enumerate(tqdm(test_loader)):
        x = batch['inputs'].to(DEVICE)
        z = model(x)
        p = torch.argmax(z, dim=-1).cpu() + 1
        l = [i.item() for i in p.flatten()]
        output.extend(l)

    return output

In [14]:
# Load model and run test
model.load_state_dict(torch.load('checkpoint.pt'))
model.to(DEVICE)
l = test(model, test_loader)

dic = {'ID': [i for i in range(len(l))], 'Prediction': l}
df = pd.DataFrame(dic)
df.to_csv('submission.csv', index=False)

# 테스트 배치사이즈 안맞을시

In [13]:
test_loader  = DataLoader(test_ds,  batch_size=2)